# 03_ETL_Project_Final_Delivery

## Proyecto Final – Pipeline ETL Completo (Spotify + LastFM)

Este notebook consolida el flujo completo del proceso ETL, desde la extracción inicial hasta la generación de reportes, utilizando la base de datos PostgreSQL alojada en Railway.  
Su estructura es equivalente al DAG de Airflow `spotify_etl_full_pipeline`, pero permite la ejecución manual paso a paso desde un entorno de desarrollo.

**Flujo general del proceso:**
- `spotify_most_streamed_2024`
- `↓ Limpieza inicial (Transform)`
- `spotify_most_streamed_clean`
- `↓ Enriquecimiento con API LastFM`
- `spotify_most_streamed_enriched`
- `↓ Validación y control de calidad`
- `spotify_most_streamed_enriched_cleaned`
- `↓ Carga final consolidada`
- `spotify_final_curated`

**Módulos utilizados:**
- `etl.extract`
- `etl.transform`
- `etl.enrich`
- `etl.validate`
- `etl.load`
- `etl.report`

In [1]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pandas as pd

load_dotenv()

DB_URI = os.getenv("DB_URI")
LASTFM_API_KEY = os.getenv("LASTFM_API_KEY")

assert DB_URI, "La variable DB_URI no está definida en el archivo .env."
assert LASTFM_API_KEY, "La variable LASTFM_API_KEY no está definida en el archivo .env."

engine = create_engine(DB_URI)
with engine.connect() as conn:
    print("Conectado correctamente a PostgreSQL en Railway.")

Conectado correctamente a PostgreSQL en Railway.


In [2]:
import os
import sys
from pathlib import Path

root_dir = Path(__file__).resolve().parent.parent if '__file__' in globals() else Path(os.getcwd()).parent
if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

print(f"Ruta raíz añadida al sys.path:\n{root_dir}")


Ruta raíz añadida al sys.path:
/Users/mariavalencia/Downloads/ETL_Class/Proyecto_3 2


## 1. Extract

En esta fase se extraen los datos iniciales desde la tabla base `spotify_most_streamed_2024` en la base de datos de Railway.  
El resultado se almacena en una nueva tabla llamada `spotify_most_streamed_clean`, que servirá como punto de partida para el proceso de limpieza.

In [3]:
from etl.extract import extract_raw_to_table

print("=== EXTRACT: Iniciando extracción ===")
extract_raw_to_table()

df_extract = pd.read_sql('SELECT * FROM spotify_most_streamed_clean', engine)
print(f"Extracción completada: {len(df_extract)} filas cargadas.")
df_extract.head(5)


=== EXTRACT: Iniciando extracción ===
[2025-11-10 00:50:13] Extrayendo datos base de Railway: spotify_most_streamed_2024
[2025-11-10 00:50:23] Tabla spotify_most_streamed_clean creada correctamente.
Extracción completada: 4600 filas cargadas.


,Track,Album Name,Artist,Release Date,ISRC,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,...,SiriusXM Spins,Deezer Playlist Count,Deezer Playlist Reach,Amazon Playlist Count,Pandora Streams,Pandora Track Stations,Soundcloud Streams,Shazam Counts,TIDAL Popularity,Explicit Track
0,MILLION DOLLAR BABY,Million Dollar Baby - Single,Tommy Richman,2024-04-26,QM24S2402528,1,725.4,390.0,30.0,196.0,...,684.0,62.0,17.0,114.0,18.0,22.0,4.0,2.0,None,0
1,Not Like Us,Not Like Us,Kendrick Lamar,2024-05-04,USUG12400910,2,545.9,323.0,28.0,174.0,...,3.0,67.0,10.0,111.0,7.0,28.0,6.0,1.0,None,1
2,i like the way you kiss me,I like the way you kiss me,Artemas,2024-03-19,QZJ842400387,3,538.4,601.0,54.0,211.0,...,536.0,136.0,36.0,172.0,5.0,5.0,7.0,5.0,None,0
3,Flowers,Flowers - Single,Miley Cyrus,2023-01-12,USSM12209777,4,444.9,2.0,269.0,136.0,...,2.0,264.0,24.0,210.0,190.0,203.0,NaN,11.0,None,0
4,Houdini,Houdini,Eminem,2024-05-31,USUG12403398,5,423.3,107.0,7.0,151.0,...,1.0,82.0,17.0,105.0,4.0,7.0,207.0,457.0,None,1


## 2. Transform

En esta etapa se realiza la limpieza y normalización de los datos.  
Se corrigen tipos de datos, se convierten columnas numéricas, se manejan valores nulos y duplicados, y se ajustan formatos de fechas.  
El resultado reemplaza la tabla `spotify_most_streamed_clean` con su versión depurada.

In [4]:
from etl.transform import transform_clean_data

print("=== TRANSFORM: Limpieza y normalización ===")
transform_clean_data()

df_transform = pd.read_sql('SELECT * FROM spotify_most_streamed_clean', engine)
print(f"Transformación completada: {len(df_transform)} registros limpios.")
df_transform.sample(5)


=== TRANSFORM: Limpieza y normalización ===
[2025-11-10 00:50:26] Leyendo 4600 filas desde spotify_most_streamed_clean


/Users/mariavalencia/Downloads/ETL_Class/Proyecto_3 2/etl/transform.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('Unknown', inplace=True)


[2025-11-10 00:50:32] Datos transformados y actualizados en spotify_most_streamed_clean.
Transformación completada: 4598 registros limpios.


,Track,Album Name,Artist,Release Date,ISRC,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,...,SiriusXM Spins,Deezer Playlist Count,Deezer Playlist Reach,Amazon Playlist Count,Pandora Streams,Pandora Track Stations,Soundcloud Streams,Shazam Counts,TIDAL Popularity,Explicit Track
559,Unstoppable,Unstoppable,Sia,2016-01-21,USRC11502934,548,65.5,1.0,166.0,75.0,...,549.0,86.0,1.0,52.0,92.0,86.0,Unknown,15.0,Unknown,0
1781,El Efecto,El Efecto,Rauw Alejandro,2019-03-15,QM4TX1940916,1,34.7,580.0,72.0,32.0,...,20.0,11.0,252.0,22.0,23.0,5.0,Unknown,2.0,Unknown,0
3486,Too Easy,Too Easy,Gunna,2021-09-24,QMCE32100757,3,23.2,133.0,69.0,7.0,...,401.0,3.0,7.0,9.0,15.0,3.0,13.0,216.0,Unknown,1
2433,O Patinho Colorido,"Bento e Totï¿½ï¿½, Vo",Bento e Totï¿,2019-11-15,BX7CP1900008,2,29.1,11.0,2.0,188.0,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,13.0,Unknown,0
3929,Mood - Remix,Mood (Remix),Zero Two,2021-06-09,VNA0R1989249,3,21.6,10.0,3.0,374.0,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,0


## 3. Enrich

En esta etapa se enriquece la información mediante la API pública de LastFM.  
Para cada combinación de artista y canción, se consultan datos complementarios como:
- Duración de la pista  
- Álbum  
- Fecha de publicación  
- Número de oyentes  
- Reproducciones  
- Etiquetas o géneros asociados

El resultado se almacena en la tabla `spotify_most_streamed_enriched`.

In [5]:
import importlib
from etl import enrich
from etl.enrich import enrich_with_lastfm
from etl.utils import get_engine, log
import pandas as pd
importlib.reload(enrich)

engine = get_engine()
log("=== ENRICH: Enriquecimiento con API LastFM ===")
enrich_with_lastfm()

df_enrich = pd.read_sql('SELECT * FROM spotify_most_streamed_enriched', engine)
log(f"Enriquecimiento completado: {len(df_enrich)} registros procesados.")
df_enrich.head(5)

[2025-11-10 00:50:33] === ENRICH: Enriquecimiento con API LastFM ===
Usando carpeta de caché: /Users/mariavalencia/ETL_cache/lastfm
[2025-11-10 00:50:36] Procesando 4488 canciones en bloques de 500...
[2025-11-10 00:50:36] Procesando: Tommy Richman - MILLION DOLLAR BABY
[2025-11-10 00:50:36] Procesando: Kendrick Lamar - Not Like Us
[2025-11-10 00:50:36] Procesando: Artemas - i like the way you kiss me
[2025-11-10 00:50:36] Procesando: Miley Cyrus - Flowers
[2025-11-10 00:50:36] Procesando: Eminem - Houdini
[2025-11-10 00:50:36] Procesando: Jack Harlow - Lovin On Me
[2025-11-10 00:50:36] Procesando: Benson Boone - Beautiful Things
[2025-11-10 00:50:36] Procesando: FloyyMenor - Gata Only
[2025-11-10 00:50:36] Procesando: MUSIC LAB JPN - Danza Kuduro - Cover
[2025-11-10 00:50:36] Procesando: Central Cee - BAND4BAND (feat. Lil Baby)
[2025-11-10 00:50:36] Procesando: Post Malone - I Had Some Help (feat. Morgan Wallen)
[2025-11-10 00:50:36] Procesando: Teddy Swims - The Door
[2025-11-10 00:5

/Users/mariavalencia/Downloads/ETL_Class/Proyecto_3 2/etl/enrich.py:165: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  bad_cells = df_temp[possible_numeric].applymap(lambda x: isinstance(x, str))


[2025-11-10 00:50:37] Guardado parcial 500/4488 registros procesados correctamente.
[2025-11-10 00:50:37] Procesando: Juliï¿½ï¿½n ï¿½ï¿½lvarez y su Norteï - Regalo De Dios
[2025-11-10 00:50:37] Procesando: Jung Kook - Never Let Go
[2025-11-10 00:50:37] Procesando: Rasha Nahas - Toyour ï¿½ï¿½ï¿
[2025-11-10 00:50:37] Procesando: Tollan Kim - Chill Vibes
[2025-11-10 00:50:37] Procesando: Bad Bunny - MIA (feat. Drake)
[2025-11-10 00:50:37] Procesando: Drake - Toosie Slide
[2025-11-10 00:50:37] Procesando: KAROL G - MIENTRAS ME CURO DEL CORA
[2025-11-10 00:50:37] Procesando: Taylor Swift - Down Bad
[2025-11-10 00:50:37] Procesando: Mustard - Ballin' (with Roddy Ricch)
[2025-11-10 00:50:37] Procesando: El Reino Infantil - La llama que Baila
[2025-11-10 00:50:37] Procesando: Luke Combs - Ain't No Love In Oklahoma (From Twisters: The Album)
[2025-11-10 00:50:37] Procesando: Doja Cat - Vegas (From the Original Motion Picture Soundtrack ELVIS)
[2025-11-10 00:50:37] Procesando: Marshmello - El Me

/Users/mariavalencia/Downloads/ETL_Class/Proyecto_3 2/etl/enrich.py:165: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  bad_cells = df_temp[possible_numeric].applymap(lambda x: isinstance(x, str))


[2025-11-10 00:50:38] Guardado parcial 1000/4488 registros procesados correctamente.
[2025-11-10 00:50:38] Procesando: LP - Lost on You
[2025-11-10 00:50:38] Procesando: Marï¿½ï¿½lia Mendo - Leï¿½
[2025-11-10 00:50:38] Procesando: Pritam - What Jhumka ? (From "Rocky Aur Rani Kii Prem Kahaani")
[2025-11-10 00:50:38] Procesando: Juice WRLD - Lean Wit Me
[2025-11-10 00:50:38] Procesando: Quavo - HOTEL LOBBY (Unc & Phew)
[2025-11-10 00:50:38] Procesando: Diljit Dosanjh - Choli Ke Peeche (From "Crew")
[2025-11-10 00:50:38] Procesando: Baba Blakes - April Rain in August Weather
[2025-11-10 00:50:38] Procesando: MAKAELA - LEATHER
[2025-11-10 00:50:38] Procesando: Lil Nas X - Panini
[2025-11-10 00:50:38] Procesando: Cole Swindell - She Had Me At Heads Carolina
[2025-11-10 00:50:38] Procesando: Taylor Swift - So Long, London
[2025-11-10 00:50:38] Procesando: Olivia Rodrigo - bad idea right?
[2025-11-10 00:50:38] Procesando: KAROL G - Ay, DiOs Mï¿½ï
[2025-11-10 00:50:38] Procesando: Olivia Rodri

...

/Users/mariavalencia/Downloads/ETL_Class/Proyecto_3 2/etl/enrich.py:165: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  bad_cells = df_temp[possible_numeric].applymap(lambda x: isinstance(x, str))
/Users/mariavalencia/Downloads/ETL_Class/Proyecto_3 2/etl/enrich.py:174: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_temp["release_date"] = pd.to_datetime(df_temp["release_date"], errors="coerce")


[2025-11-10 00:51:01] Guardado parcial 4000/4488 registros procesados correctamente.
[2025-11-10 00:51:01] Procesando: Badshah - Aashiq Awaara (feat. Sunidhi Chauhan)
[2025-11-10 00:51:01] Procesando: IVE - Off The Record
[2025-11-10 00:51:01] Procesando: Mï¿½ï¿½ne - BABY SAID
[2025-11-10 00:51:01] Procesando: Pop Smoke - Gangstas
[2025-11-10 00:51:01] Procesando: Drake - Too Good
[2025-11-10 00:51:01] Procesando: Rod Wave - Alone
[2025-11-10 00:51:01] Procesando: Gracie's Corner - Veggie Dance - Remix
[2025-11-10 00:51:01] Procesando: Central Cee - One Up
[2025-11-10 00:51:01] Procesando: blackbear - idfc
[2025-11-10 00:51:01] Procesando: HCTM - Revisando El Celu
[2025-11-10 00:51:01] Procesando: Anirudh Ravichander - Not Ramaiya Vastavaiya (From "Jawan")
[2025-11-10 00:51:01] Procesando: $uicideboy$ - Are You Going to See the Rose in the Vase, or the Dust on the Table
[2025-11-10 00:51:01] Procesando: Rihanna - Only Girl (In The World)
[2025-11-10 00:51:01] Procesando: Khalid - OTW
[

/Users/mariavalencia/Downloads/ETL_Class/Proyecto_3 2/etl/enrich.py:165: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  bad_cells = df_temp[possible_numeric].applymap(lambda x: isinstance(x, str))


[2025-11-10 00:51:16] Guardado parcial 4488/4488 registros procesados correctamente.
[2025-11-10 00:51:16]  Enriquecimiento completado correctamente.
[2025-11-10 00:51:23] Enriquecimiento completado: 95225 registros procesados.


,Track,Album Name,Artist,Release Date,ISRC,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,...,Soundcloud Streams,Shazam Counts,TIDAL Popularity,Explicit Track,duration_ms,album,release_date,listeners,playcount,tags
0,MILLION DOLLAR BABY,Million Dollar Baby - Single,Tommy Richman,2024-04-26,QM24S2402528,1,725.4,390.0,30.0,196.0,...,4.0,2.0,0.0,0,155000.0,Million Dollar Baby,"15 May 2024, 23:46",1283198,15463688,"melodic, instrumental, danceable"
1,Not Like Us,Not Like Us,Kendrick Lamar,2024-05-04,USUG12400910,2,545.9,323.0,28.0,174.0,...,6.0,1.0,0.0,1,275000.0,Not Like Us,"05 May 2024, 21:35",1814693,27870490,"Diss, Hip-Hop, diss track"
2,i like the way you kiss me,I like the way you kiss me,Artemas,2024-03-19,QZJ842400387,3,538.4,601.0,54.0,211.0,...,7.0,5.0,0.0,0,143000.0,i like the way you kiss me,"01 Apr 2024, 04:10",1112409,15325912,"coldwave, post-punk, synthpop"
3,Flowers,Flowers - Single,Miley Cyrus,2023-01-12,USSM12209777,4,444.9,2.0,269.0,136.0,...,0.0,11.0,0.0,0,200000.0,Flowers,"13 Jan 2023, 01:37",1125177,14959395,"pop, 2023, Disco"
4,Houdini,Houdini,Eminem,2024-05-31,USUG12403398,5,423.3,107.0,7.0,151.0,...,207.0,457.0,0.0,1,227000.0,Houdini,"31 May 2024, 13:58",632179,6419574,"2024, hip hop, rap"


## 4. Validate

En esta fase se valida la integridad del dataset enriquecido.  
Se eliminan duplicados, se corrigen valores faltantes y se aplican verificaciones básicas de consistencia.  
La tabla resultante se guarda como `spotify_most_streamed_enriched_cleaned`.

In [6]:
from etl.validate import validate_enriched_data

print("=== VALIDATE: Validación de consistencia y estructura ===")
validate_enriched_data()

df_validate = pd.read_sql('SELECT * FROM spotify_most_streamed_enriched_cleaned', engine)
print(f"Validación completada: {len(df_validate)} registros válidos.")
df_validate.head(5)


=== VALIDATE: Validación de consistencia y estructura ===
[2025-11-10 00:51:49] Validando 97921 registros...
Guardando resumen en: /Users/mariavalencia/ETL_reports/great_expectations/summary_20251110_005149.txt
[2025-11-10 00:51:55] Validación completada. Guardado 'spotify_most_streamed_enriched_cleaned' y resumen en /Users/mariavalencia/ETL_reports/great_expectations/summary_20251110_005149.txt
Validación completada: 3571 registros válidos.


,Track,Album Name,Artist,Release Date,ISRC,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,...,Soundcloud Streams,Shazam Counts,TIDAL Popularity,Explicit Track,duration_ms,album,release_date,listeners,playcount,tags
0,MILLION DOLLAR BABY,Million Dollar Baby - Single,Tommy Richman,2024-04-26,QM24S2402528,1,725.4,390.0,30.0,196.0,...,4.0,2.0,0.0,0,155000.0,Million Dollar Baby,"15 May 2024, 23:46",1283198,15463688,"melodic, instrumental, danceable"
1,Not Like Us,Not Like Us,Kendrick Lamar,2024-05-04,USUG12400910,2,545.9,323.0,28.0,174.0,...,6.0,1.0,0.0,1,275000.0,Not Like Us,"05 May 2024, 21:35",1814693,27870490,"Diss, Hip-Hop, diss track"
2,i like the way you kiss me,I like the way you kiss me,Artemas,2024-03-19,QZJ842400387,3,538.4,601.0,54.0,211.0,...,7.0,5.0,0.0,0,143000.0,i like the way you kiss me,"01 Apr 2024, 04:10",1112409,15325912,"coldwave, post-punk, synthpop"
3,Flowers,Flowers - Single,Miley Cyrus,2023-01-12,USSM12209777,4,444.9,2.0,269.0,136.0,...,0.0,11.0,0.0,0,200000.0,Flowers,"13 Jan 2023, 01:37",1125177,14959395,"pop, 2023, Disco"
4,Houdini,Houdini,Eminem,2024-05-31,USUG12403398,5,423.3,107.0,7.0,151.0,...,207.0,457.0,0.0,1,227000.0,Houdini,"31 May 2024, 13:58",632179,6419574,"2024, hip hop, rap"


## 5. Load

Una vez validados los datos, se consolidan en una tabla final denominada `spotify_final_curated`.  
Esta tabla actúa como el dataset definitivo del proyecto, listo para su uso en análisis o visualización.

In [7]:
from etl.load import finalize_curated_table

print("=== LOAD: Consolidación final del modelo ===")
finalize_curated_table()

df_load = pd.read_sql('SELECT * FROM spotify_final_curated', engine)
print(f"Carga final completada: {len(df_load)} registros consolidados.")
df_load.head(5)

=== LOAD: Consolidación final del modelo ===
[2025-11-10 00:51:56] Leyendo tabla validada: spotify_most_streamed_enriched_cleaned
[2025-11-10 00:52:08] Tabla final 'spotify_final_curated' actualizada exitosamente (3571 filas).
Carga final completada: 3571 registros consolidados.


,Track,Album Name,Artist,Release Date,ISRC,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,...,Soundcloud Streams,Shazam Counts,TIDAL Popularity,Explicit Track,duration_ms,album,release_date,listeners,playcount,tags
0,MILLION DOLLAR BABY,Million Dollar Baby - Single,Tommy Richman,2024-04-26,QM24S2402528,1,725.4,390.0,30.0,196.0,...,4.0,2.0,0.0,0,155000.0,Million Dollar Baby,"15 May 2024, 23:46",1283198,15463688,"melodic, instrumental, danceable"
1,Not Like Us,Not Like Us,Kendrick Lamar,2024-05-04,USUG12400910,2,545.9,323.0,28.0,174.0,...,6.0,1.0,0.0,1,275000.0,Not Like Us,"05 May 2024, 21:35",1814693,27870490,"Diss, Hip-Hop, diss track"
2,i like the way you kiss me,I like the way you kiss me,Artemas,2024-03-19,QZJ842400387,3,538.4,601.0,54.0,211.0,...,7.0,5.0,0.0,0,143000.0,i like the way you kiss me,"01 Apr 2024, 04:10",1112409,15325912,"coldwave, post-punk, synthpop"
3,Flowers,Flowers - Single,Miley Cyrus,2023-01-12,USSM12209777,4,444.9,2.0,269.0,136.0,...,0.0,11.0,0.0,0,200000.0,Flowers,"13 Jan 2023, 01:37",1125177,14959395,"pop, 2023, Disco"
4,Houdini,Houdini,Eminem,2024-05-31,USUG12403398,5,423.3,107.0,7.0,151.0,...,207.0,457.0,0.0,1,227000.0,Houdini,"31 May 2024, 13:58",632179,6419574,"2024, hip hop, rap"


## 6. Report

En la etapa final se generan los reportes estadísticos y gráficos del proyecto.  
Estos reportes incluyen:
- Tablas agregadas y resúmenes (en formato CSV)
- Visualizaciones de distribución y correlación
- Reportes de validación automática

Todos los resultados se almacenan en la carpeta `/reports/`.

In [8]:
import os
from etl.report import generate_reports

print("=== REPORT: Generación de reportes ===")
generate_reports()

current_dir = os.getcwd()
base_dir = os.path.abspath(os.path.join(current_dir, "..")) if "notebooks" in current_dir else current_dir
reports_dir = os.path.join(base_dir, "reports")

print(f"\nArchivos generados en el directorio '{reports_dir}':\n")
for root, _, files in os.walk(reports_dir):
    for file in files:
        print(os.path.join(root, file))


=== REPORT: Generación de reportes ===


/Users/mariavalencia/Downloads/ETL_Class/Proyecto_3 2/etl/report.py:35: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=top_artists.values, y=top_artists.index, palette="viridis")


[2025-11-10 00:52:16] Reportes y figuras generados correctamente.

Archivos generados en el directorio '/Users/mariavalencia/Downloads/ETL_Class/Proyecto_3 2/reports':

/Users/mariavalencia/Downloads/ETL_Class/Proyecto_3 2/reports/figures/03_duration_popularity.png
/Users/mariavalencia/Downloads/ETL_Class/Proyecto_3 2/reports/figures/02_streams_evolution.png
/Users/mariavalencia/Downloads/ETL_Class/Proyecto_3 2/reports/figures/01_top_artists.png
/Users/mariavalencia/Downloads/ETL_Class/Proyecto_3 2/reports/figures/04_multichannel_correlation.png
/Users/mariavalencia/Downloads/ETL_Class/Proyecto_3 2/reports/great_expectations/summary_20251109_084019.txt
/Users/mariavalencia/Downloads/ETL_Class/Proyecto_3 2/reports/great_expectations/summary_20250927_151352.txt
/Users/mariavalencia/Downloads/ETL_Class/Proyecto_3 2/reports/great_expectations/summary_20251109_153325.txt
/Users/mariavalencia/Downloads/ETL_Class/Proyecto_3 2/reports/great_expectations/validation_report_20250927_151352.json
/

## 7. Resumen de estructura en la base de datos

Esta sección muestra un resumen de las tablas del proyecto en la base de datos PostgreSQL, junto con su número de filas y columnas.

In [9]:
from sqlalchemy import inspect

inspector = inspect(engine)

tables = [
    "spotify_most_streamed_2024",
    "spotify_most_streamed_clean",
    "spotify_most_streamed_enriched",
    "spotify_most_streamed_enriched_cleaned",
    "spotify_final_curated"
]

print("=== Resumen de tablas en PostgreSQL (Railway) ===\n")
for table in tables:
    try:
        cols = [c['name'] for c in inspector.get_columns(table)]
        count = pd.read_sql(f'SELECT COUNT(*) FROM "{table}"', engine).iloc[0,0]
        print(f"{table:<45} | Filas: {count:<6} | Columnas: {len(cols)}")
    except Exception as e:
        print(f"{table:<45} | Error: {e}")

=== Resumen de tablas en PostgreSQL (Railway) ===

spotify_most_streamed_2024                    | Filas: 4600   | Columnas: 29
spotify_most_streamed_clean                   | Filas: 4598   | Columnas: 29
spotify_most_streamed_enriched                | Filas: 100667 | Columnas: 35
spotify_most_streamed_enriched_cleaned        | Filas: 3571   | Columnas: 35
spotify_final_curated                         | Filas: 3571   | Columnas: 35


## Conclusión

Este notebook ejecuta el flujo completo del pipeline ETL del proyecto final.  
Todos los pasos son idempotentes y se pueden ejecutar de forma independiente o secuencial.  

El modelo final, `spotify_final_curated`, representa la consolidación de los datos más relevantes, limpios y verificados del ecosistema Spotify + LastFM.  
Las salidas del pipeline se encuentran en la carpeta `reports/` y las tablas intermedias permanecen almacenadas en la base de datos PostgreSQL alojada en Railway.